Pseudocode for Permutation Core Butson-Type Complex Hadamard Matrices:

* Select dimension $n$ for matrix
* Generate multiples of $\Phi_n(x)$ with sum of coefficient magnitudes equal to $n$
* Normalize so that they're all positive coefficients by multiplying by $x^{n/2}$ for the negative ones

In [7]:
# type: ignore

import itertools

# -----------------------------
# 1. Generate positive-coefficient L1-bounded rows
# -----------------------------
def generate_L1_vectors_positive(d, n):
    """
    Yield tuples (c0,...,cd-1) with:
      - all ci >= 0
      - sum(ci) == n
      - c0 >= 1 (dephasing)
    """
    coeffs = [0]*d

    def backtrack(i, remaining):
        if i == d:
            if remaining == 0 and coeffs[0] >= 1:
                yield tuple(coeffs)
            return
        start = 1 if i == 0 else 0
        for val in range(start, remaining+1):
            coeffs[i] = val
            yield from backtrack(i+1, remaining - val)
            coeffs[i] = 0

    yield from backtrack(0, n)

# -----------------------------
# 2. Orthogonality check using polynomials on the fly
# -----------------------------
def rows_orthogonal(row1, row2, d):
    R = ZZ['x']
    x = R.gen()
    Phi = cyclotomic_polynomial(d)
    # build polynomials on the fly
    p1 = sum(c*x^i for i, c in enumerate(row1))
    p2 = sum(c*x^i for i, c in enumerate(row2))
    return (p1 * p2) % Phi == 0

# -----------------------------
# 3. Recursive construction of permutation-core CHMs
# -----------------------------
def build_permutation_core_CHMs(core_coeffs, n):
    """
    core_coeffs: list of integers (first entry >= 1)
    d: size of the CHM
    Returns: list of matrices (each matrix is a list of rows, each row is a list)
    """
    first_row = [1]*n
    fixed_first = core_coeffs[0]
    tail = core_coeffs[1:]
    results = []

    def backtrack(existing_rows):
        if len(existing_rows) == n:
            results.append([row[:] for row in existing_rows])
            return
        for perm in itertools.permutations(tail):
            candidate_row = [fixed_first] + list(perm)
            if all(rows_orthogonal(candidate_row, row, n) for row in existing_rows):
                existing_rows.append(candidate_row)
                backtrack(existing_rows)
                existing_rows.pop()

    backtrack([first_row])
    return results

# -----------------------------
# 4. Example usage
# -----------------------------
d = 4   # CHM size / degree
n = 3   # sum of coefficients for core row

for coeffs in generate_L1_vectors_positive(d, n):
    chms = build_permutation_core_CHMs(list(coeffs), n)
    if chms:
        print(f"Core coefficients: {coeffs}")
        for mat in chms:
            for row in mat:
                print(row)
            print("---")


Core coefficients: (1, 0, 1, 1)
[1, 1, 1]
[1, 0, 1, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 0, 1, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 0, 1, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 0, 1, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 0, 1, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 0, 1, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 0, 1, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 0, 1, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
---
Core coefficients: (1, 1, 0, 1)
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 0, 1]
[1, 1, 1, 0]
---
[1, 1, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
---
[1, 1, 1]
[1, 1,